# INGESTION DE DATOS TRANSACCIONALES

In [1]:
## CONFIGURACION DE AWS RDS

In [29]:
# Librerias necesarias
import boto3
import pandas as pd
import numpy as np
import psycopg2
import configparser
from faker import Faker
from sqlalchemy import create_engine
from sqlalchemy import text

import warnings
import random
warnings.filterwarnings('ignore')


ImportError: cannot import name 'session' from 'sqlalchemy' (/Users/roolzpolo/anaconda3/lib/python3.11/site-packages/sqlalchemy/__init__.py)

In [3]:
# Documento privado de claves
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [4]:
# Identificacion con AWS
aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get('IAM','ACCESS_KEY'),
                             aws_secret_access_key=config.get('IAM','SECRET_ACCESS_KEY'),
                             region_name='us-east-1')

In [5]:
#Verificacion de instancias creadas anteriormente.

rds_instances_ids = []
aws_response = aws_rds_conn.describe_db_instances() 

for response in aws_response['DBInstances']:
    rds_instances_ids.append(response['DBInstanceIdentifier'])

print(f"Instancias Disponibles : {rds_instances_ids}")

Instancias Disponibles : ['banco-transactional']


In [6]:
#creacion de la base de datos

try:
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier = config.get('TRANSACC','DB_INSTANCE_ID'),
        DBName = config.get('TRANSACC','DB_NAME'),
        DBInstanceClass = 'db.t3.micro',
        Engine = 'postgres',
        MasterUsername = config.get('TRANSACC','DB_USER'),
        MasterUserPassword = config.get('TRANSACC','DB_PASSWORD'),
        Port = int(config.get('TRANSACC','DB_PORT')),
        PubliclyAccessible = True,
        VpcSecurityGroupIds = [config.get('VPC','SECURITY_GROUP')],
        AllocatedStorage = 15
               )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print('La instancia ya existe')
except Exeption as ex:
    print('ERROR', ex)

La instancia ya existe


### Obtencion del hostname de las instancias

In [7]:
# codigo para obtener el hostname

try: 
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier = config.get('TRANSACC','DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print('Error', ex)

banco-transactional.c1m44mee4c6c.us-east-1.rds.amazonaws.com


### Conexion de base de datos desde python

In [8]:
import sql_queries

In [9]:
sql_queries.DDL_QUERY

'\nCREATE TABLE IF NOT EXISTS categoria (\n    idcategoria SERIAL PRIMARY KEY,\n    nombre VARCHAR(50) NOT NULL,\n    descripcion VARCHAR(255),\n    estado BOOLEAN NOT NULL\n);\n\nCREATE TABLE IF NOT EXISTS rol (\n    idrol SERIAL PRIMARY KEY,\n    nombre VARCHAR(30) NOT NULL,\n    descripcion VARCHAR(255),\n    estado BOOLEAN NOT NULL\n);\n\nCREATE TABLE IF NOT EXISTS usuario (\n    idusuario SERIAL PRIMARY KEY,\n    idrol INTEGER NOT NULL,\n    nombre VARCHAR(100) NOT NULL,\n    tipo_documento VARCHAR(20),\n    num_documento VARCHAR(20),\n    direccion VARCHAR(70),\n    telefono VARCHAR(20),\n    email VARCHAR(50) NOT NULL,\n    clave VARCHAR(50) NOT NULL,\n    estado BOOLEAN NOT NULL,\n    CONSTRAINT fk_usuario_rol FOREIGN KEY (idrol) REFERENCES rol (idrol)\n);\n\nCREATE TABLE IF NOT EXISTS persona (\n    idpersona SERIAL PRIMARY KEY,\n    tipo_persona VARCHAR(20) NOT NULL,\n    nombre VARCHAR(100) NOT NULL,\n    tipo_documento VARCHAR(20),\n    num_documento VARCHAR(20),\n    direc

In [10]:
#creacion de database de datos
try:
    db_pg_conn = psycopg2.connect(
                    dbname = config.get('TRANSACC','DB_NAME'),
                    user = config.get('TRANSACC','DB_USER'),
                    password = config.get('TRANSACC','DB_PASSWORD'),
                    host = RDS_HOSTNAME,
                    port = config.get('TRANSACC','DB_PORT')
    )
    cursor = db_pg_conn.cursor()
    cursor.execute(sql_queries.DDL_QUERY)
    db_pg_conn.commit()
    print('Base de datos creada exitosamente')

except Exception as ex:
    print('ERROR', ex)

Base de datos creada exitosamente


## Generacion de datos con Faker

In [13]:
# Carga de librerias nuevamente
from faker import Faker
import pandas as pd
from sqlalchemy import create_engine
from configparser import ConfigParser

In [16]:
# # Conectar a la base de datos PostgreSQL usando SQLAlchemy

driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOSTNAME}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""
engine = create_engine(driver)
# insertData2SQL(data_tipo_transacciones, 'tipo_transacciones', driver)

NameError: name 'insertData2SQL' is not defined

In [17]:
#ingreso de data para 'CATEGORIA'
# Inicializar Faker
fake = Faker()

# Función para generar datos para la tabla 'categoria'
def generate_categoria_data(n):
    data = []
    for _ in range(n):
        data.append({
            "nombre": fake.word(),
            "descripcion": fake.text(max_nb_chars=255),
            "estado": fake.boolean()
        })
    return data

# Insertar datos a la base de datos
def insertData2SQL(data, table_name, engine):
    df_data = pd.DataFrame(data)
    try:
        response = df_data.to_sql(table_name, engine, index=False, if_exists='append')
        print(f"Se han insertado {len(df_data)} nuevos registros en la tabla {table_name}.")
    except Exception as ex:
        print(f"Error al insertar datos: {ex}")

# Generar 1000 registros de datos para 'categoria'
data_categoria = generate_categoria_data(1000)

# Crear motor SQLAlchemy
engine = create_engine(driver)

# Insertar datos generados en la tabla 'categoria'
insertData2SQL(data_categoria, 'categoria', driver)

Se han insertado 1000 nuevos registros en la tabla categoria.


In [18]:
# INGRESO DE DATA PARA ROL
# Inicializar Faker
fake = Faker()

# Función para generar datos para la tabla 'rol'
def generate_rol_data(n):
    data = []
    for _ in range(n):
        data.append({
            "nombre": fake.job()[:30],  # Genera nombres de roles basados en tipos de trabajos
            "descripcion": fake.text(max_nb_chars=255),  # Descripción falsa
            "estado": fake.boolean()  # Estado aleatorio
        })
    return data

# Insertar datos a la base de datos
def insertData2SQL(data, table_name, engine):
    df_data = pd.DataFrame(data)
    try:
        response = df_data.to_sql(table_name, engine, index=False, if_exists='append')
        print(f"Se han insertado {len(df_data)} nuevos registros en la tabla {table_name}.")
    except Exception as ex:
        print(f"Error al insertar datos: {ex}")

# Generar 1000 registros de datos para 'categoria'
data_rol = generate_rol_data(1000)

# Crear motor SQLAlchemy
engine = create_engine(driver)

# Insertar datos generados en la tabla 'categoria'
insertData2SQL(data_rol, 'rol', driver)

Se han insertado 1000 nuevos registros en la tabla rol.


In [30]:
#INGRESO DE DATA PARA USUARIO
# Inicializar Faker
fake = Faker()

# Crear el motor SQLAlchemy
engine = create_engine(driver)
Session = sessionmaker(bind=engine)
session = Session()

# Función para obtener los roles existentes
def get_existing_roles(session):
    results = session.execute(text("SELECT idrol FROM rol"))
    existing_roles = [row[0] for row in results]
    return existing_roles

# Obtener los roles existentes
existing_roles = get_existing_roles(session)

# Función para generar datos para la tabla 'usuario'
def generate_usuario_data(n, existing_roles):
    data = []
    for _ in range(n):
        data.append({
            "idrol": fake.random.choice(existing_roles),
            "nombre": fake.name()[:100],
            "tipo_documento": fake.random.choice(['DNI', 'PAS', 'RUC', 'CE'])[:20],
            "num_documento": fake.bothify(text='########')[:20],
            "direccion": fake.address()[:70],
            "telefono": fake.phone_number()[:20],
            "email": fake.email()[:50],
            "clave": fake.password(length=10, special_chars=True, digits=True, upper_case=True, lower_case=True)[:50],
            "estado": fake.boolean()
        })
    return data

# Función para insertar datos en la base de datos
def insertData2SQL(data, table_name, engine):
    df_data = pd.DataFrame(data)
    try:
        response = df_data.to_sql(table_name, engine, index=False, if_exists='append')
        print(f"Se han insertado {len(df_data)} nuevos registros en la tabla {table_name}.")
    except Exception as ex:
        print(f"Error al insertar datos: {ex}")

# Generar 1000 registros de datos para 'usuario'
data_usuario = generate_usuario_data(1000, existing_roles)

# Insertar datos generados en la tabla 'usuario'
insertData2SQL(data_usuario, 'usuario', engine)

# Cerrar la sesión
session.close()

Se han insertado 1000 nuevos registros en la tabla usuario.


In [31]:
# INGRESO DE DATA PARA PERSONA
# Inicializar Faker
fake = Faker()

# Función para generar datos para la tabla 'persona'
def generate_persona_data(n):
    data = []
    for _ in range(n):
        data.append({
            "tipo_persona": fake.random_element(elements=('Natural', 'Jurídica')),
            "nombre": fake.name(),
            "tipo_documento": fake.random_element(elements=('DNI', 'PAS', 'RUC', 'CE')),
            "num_documento": fake.bothify(text='?#######', letters='ABCDEFGHJKLMNPQRSTUVWXYZ'),
            "direccion": fake.street_address(),
            "telefono": fake.phone_number(),
            "email": fake.email()
        })
    return data

# Insertar datos a la base de datos
def insertData2SQL(data, table_name, engine):
    df_data = pd.DataFrame(data)
    try:
        response = df_data.to_sql(table_name, engine, index=False, if_exists='append')
        print(f"Se han insertado {len(df_data)} nuevos registros en la tabla {table_name}.")
    except Exception as ex:
        print(f"Error al insertar datos: {ex}")

# Generar 1000 registros de datos para 'persona'
data_persona = generate_persona_data(1000)

# Crear motor SQLAlchemy
engine = create_engine(driver)

# Insertar datos generados en la tabla 'persona'
insertData2SQL(data_persona, 'persona', driver)

Error al insertar datos: (psycopg2.errors.StringDataRightTruncation) value too long for type character varying(20)

[SQL: INSERT INTO persona (tipo_persona, nombre, tipo_documento, num_documento, direccion, telefono, email) VALUES (%(tipo_persona__0)s, %(nombre__0)s, %(tipo_documento__0)s, %(num_documento__0)s, %(direccion__0)s, %(telefono__0)s, %(email__0)s), (%(tipo_p ... 144987 characters truncated ... ipo_documento__999)s, %(num_documento__999)s, %(direccion__999)s, %(telefono__999)s, %(email__999)s)]
[parameters: {'email__0': 'matthew17@example.org', 'direccion__0': '9530 Crawford Crossing', 'num_documento__0': 'T6659534', 'nombre__0': 'Chelsea Watson', 'tipo_persona__0': 'Natural', 'tipo_documento__0': 'CE', 'telefono__0': '624-257-9136', 'email__1': 'sharirichmond@example.com', 'direccion__1': '238 Lauren Track Apt. 610', 'num_documento__1': 'R1230865', 'nombre__1': 'Kylie Wolfe', 'tipo_persona__1': 'Natural', 'tipo_documento__1': 'RUC', 'telefono__1': '5957857066', 'email__2': 